In [ ]:
import logging
from galvasr2.align.spark.align_lib import load_audio_id_text_id_mapping, load_transcripts


spark = SparkSession.builder \
                      .master("local[*]") \
                      .appName("NER") \
                      .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
                      .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")\
                      .config("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")\
                      .config("spark.driver.memory", "40g")\
                      .config("spark.executor.memory", "40g")\
                      .config("spark.rpc.askTimeout", "480s")\
                      .config("spark.executor.heartbeatInterval", "20000ms")\
                      .config("spark.eventLog.enabled", "true")\
                      .getOrCreate()
# spark.sparkContext.setLogLevel("INFO") # "ALL" for very verbose logging                                                                                                                                   
logging.getLogger("py4j").setLevel(logging.ERROR)
catalogue_df = load_audio_id_text_id_mapping(spark, "gs://the-peoples-speech-west-europe/archive_org/Mar_7_2021/CC_BY_SA_EXPANDED_LICENSES_FILTERED_ACCESS.jsonl.gz")
training_sample_rows = catalogue_df.collect()

# Comment this out to load everything. It might takes ~15 minute, in my experience, on an 8 core machine.
training_sample_rows = training_sample_rows[:100]
transcripts_df = load_transcripts(spark,
                                  "gs://the-peoples-speech-west-europe/archive_org/Mar_7_2021/CC_BY_SA_EXPANDED_LICENSES_FILTERED_ACCESS",
                                  training_sample_rows)
transcripts_pdf = transcripts_df.toPandas()

In [6]:
transcripts_pdf.count()

transcript          100
text_document_id    100
dtype: int64